## 12/11
- 肥満、高脂血症、高血糖、脂質異常（MetSにおいて）を判定
- メタボの判定


In [ ]:
# import master_code as mc
import numpy as np
import pandas as pd
main = pd.read_csv('/Volumes/USBDISK/AHHDC/git/html/dummy2.csv', encoding = 'cp932', index_col = 0)
#　変数なし

df = main.copy()

def mets_judge(df):   # 変数最小
    #腹囲判定
    df['腹囲_'] = (df['腹囲'].mask((df['性別'] == '男') & (df['腹囲'] >= 85), True) # 男性       #どちらもデータがあってTrueの場合True       
                            .mask((df['性別'] == '女') & (df['腹囲'] >= 90), True) # 女性    　
                            .mask((df['性別'] == '男') & (df['腹囲'] < 85), False)  # 男性      #どちらもデータがあってFalseの場合False
                            .mask((df['性別'] == '女') & (df['腹囲'] < 90), False)  # 女性
                            .mask((df['性別'].isnull()) | (df['腹囲'].isnull()), np.nan)    #性別 or 腹囲がない場合nan
       )

    #血圧判定
    df['血圧_'] = (df['収縮期血圧'] .mask((df['収縮期血圧'] >= 130) | (df['拡張期血圧'] >= 85), True)      # どちらかがTrueの場合True
                              .mask((df['収縮期血圧'] < 130) & (df['拡張期血圧'] < 85), False)            # どちらもFalseの場合False
                              .mask((~(df['収縮期血圧'] >= 130) & ~(df['拡張期血圧'] >= 85))                    # どちらもFalseかnanであって
                                  & ((df['収縮期血圧'].isnull()) | (df['拡張期血圧'].isnull())), np.nan)      # どちらもnanの場合nan
       )

    #血糖判定
    df['血糖_'] = (df['空腹時血糖'].mask((df['空腹時血糖'] >= 110) | (df['HbA1c'] >= 6.0), True)          # どちらかがTrueの場合True
                              .mask((df['空腹時血糖'] < 110) & (df['HbA1c'] < 6.0), False)                # どちらもFalseの場合False
                              .mask((~(df['空腹時血糖'] >= 110) & ~(df['HbA1c'] >= 6.0))                   # どちらもFalseかnanであって
                                  & ((df['空腹時血糖'].isnull()) | (df['HbA1c'].isnull())), np.nan)      # どちらもnanの場合nan
       )

    #脂質判定
    df['脂質_'] = (df['中性脂肪'].mask((df['中性脂肪'] >= 150) | (df['HDL'] <= 40), True)           # どちらかがTrueの場合True
                               .mask((df['中性脂肪'] < 150) & (df['HDL'] > 40), False)               # どちらもFalseの場合False
                               .mask((~(df['中性脂肪'] >= 150) & ~(df['HDL'] <= 40))                 # どちらもFalseかnanであって     
                                  & ((df['中性脂肪'].isnull()) | (df['HDL'].isnull())), np.nan)      # どちらもnanの場合nan
       )

    df['MetS'] = (df['腹囲'] 
                .mask((df['腹囲_'].isnull()) |                                                           # 腹囲がない or                      # 4-1
                     ((df['腹囲_'] == True) &                                                            # 腹囲がある &
                     ((df['血圧_'].isnull()) & (df['血糖_'].isnull()) & (df['脂質_'].isnull()) |          # 3項目がない(nanがある) or           # 4-2 
                      (df['血圧_'] + df['血糖_'] + df['脂質_'] == 1) &                                    # 1つの項目が異常 &
                     ((df['血圧_'].isnull()) | (df['血糖_'].isnull()) | (df['脂質_'].isnull()))))         # nanがある                         # 4-3
                       , 'N/A')                                                                         # 判定なし         

                .mask((df['腹囲_'] == False) |                                                           # 腹囲正常　or                       # 3-1
                     ((df['腹囲_'] == True) &                                                            # 腹囲異常　&
                      (df['血圧_'] + df['血糖_'] + df['脂質_'] == 0) &                                    # 3項目が正常 &
                      (df['血圧_'].notnull()) & (df['血糖_'].notnull()) & (df['脂質_'].notnull()))        # 3項目がある(nanがない)              # 3-2
                       , 'not_MetS')                                                                    # 腹囲がない & 異常なし

                .mask((df['腹囲_'] == True) &                                                            # 腹囲異常　& 
                      (df['血圧_'] + df['血糖_'] + df['脂質_'] == 1) &                                    # 1つの項目が異常 &
                      (df['血圧_'].notnull()) & (df['血糖_'].notnull()) & (df['脂質_'].notnull())         # 3項目がある(nanがない)              # 2-1
                       , 's/o_MetS')                                                                    # 腹囲がある & 1つの項目が異常
                  
                .mask((df['腹囲_'] == True) & ((df['血圧_'] + df['血糖_'] + df['脂質_'] >= 2))             # 1-1, 1-2
                       , 'MetS')                                                                        # 腹囲がある & 2つ以上の項目が異常                                                                     
       )
    
    df_metS = df[df['MetS'] == 'MetS']
    df_so_metS = df[df['MetS'] == 's/o_MetS']
    df_non_metS = df[df['MetS'] == 'not_MetS']
    df_na_metS = df[df['MetS'] == 'N/A']
    return df_metS, df_so_metS, df_non_metS, df_na_metS

In [ ]:
df = main.copy()
metS_facter = [
    ['性別', '腹囲'],               # 腹囲
    ['収縮機血圧', '拡張期血圧'],     # 血圧
    ['空腹時血糖', 'HbA1c'],        # 血糖
    ['中性脂肪', 'HDL']             # 脂質
]

def mets_judge(df, metS_facter = [['性別', '腹囲'], ['収縮期血圧', '拡張期血圧'], ['空腹時血糖', 'HbA1c'], ['中性脂肪', 'HDL']]):   #修正前
    #腹囲判定
    df['腹囲_'] = (df[metS_facter[0][1]].mask((df[metS_facter[0][0]] == 1) & (df[metS_facter[0][1]] >= 85), True) # 男性       #どちらもデータがあってTrueの場合True
                            .mask((df[metS_facter[0][0]] == 0) & (df[metS_facter[0][1]] >= 90), True) # 女性    　
                            .mask((df[metS_facter[0][0]] == 1) & (df[metS_facter[0][1]] < 85), False)  # 男性      #どちらもデータがあってFalseの場合False
                            .mask((df[metS_facter[0][0]] == 0) & (df[metS_facter[0][1]] < 90), False)  # 女性
                            .mask((df[metS_facter[0][0]].isnull()) | (df[metS_facter[0][1]].isnull()), np.nan)    #性別 or 腹囲がない場合nan
    )

    #血圧判定
    df['血圧_'] = (df[metS_facter[1][0]].mask((df[metS_facter[1][0]] >= 130) | (df[metS_facter[1][1]] >= 85), True)      # どちらかがTrueの場合True
                            .mask((df[metS_facter[1][0]] < 130) & (df[metS_facter[1][1]] < 85), False)            # どちらもFalseの場合False
                            .mask((~(df[metS_facter[1][0]] >= 130) & ~(df[metS_facter[1][1]] >= 85))                    # どちらもFalseかnanであって
                                  & ((df[metS_facter[1][0]].isnull()) | (df[metS_facter[1][1]].isnull())), np.nan)      # どちらもnanの場合nan
    )

    #血糖判定
    df['血糖_'] = (df[metS_facter[2][0]].mask((df[metS_facter[2][0]] >= 110) | (df[metS_facter[2][1]] >= 6.0), True)          # どちらかがTrueの場合True
                            .mask((df[metS_facter[2][0]] < 110) & (df[metS_facter[2][1]] < 6.0), False)                # どちらもFalseの場合False
                            .mask((~(df[metS_facter[2][0]] >= 110) & ~(df[metS_facter[2][1]] >= 6.0))                   # どちらもFalseかnanであって
                                  & ((df[metS_facter[2][0]].isnull()) | (df[metS_facter[2][1]].isnull())), np.nan)      # どちらもnanの場合nan
    )

    #脂質判定
    df['脂質_'] = (df[metS_facter[3][0]].mask((df[metS_facter[3][0]] >= 150) | (df[metS_facter[3][1]] <= 40), True)           # どちらかがTrueの場合True
                            .mask((df[metS_facter[3][0]] < 150) & (df[metS_facter[3][1]] > 40), False)               # どちらもFalseの場合False
                            .mask((~(df[metS_facter[3][0]] >= 150) & ~(df[metS_facter[3][1]] <= 40))                 # どちらもFalseかnanであって     
                                  & ((df[metS_facter[3][0]].isnull()) | (df[metS_facter[3][1]].isnull())), np.nan)      # どちらもnanの場合nan
    )

    df['MetS'] = (df['腹囲']
                .mask((df['腹囲_'] == True) & ((df['血圧_'] + df['血糖_'] + df['脂質_'] >= 2))           
                       , 'MetS')                                                                        # 腹囲がある & 2つ以上の項目が異常

                .mask((df['腹囲_'] == True) &                                                            # 腹囲異常　& 
                      (df['血圧_'] + df['血糖_'] + df['脂質_'] == 1) &                                    # 1つの項目が異常 &
                      (df['血圧_'].notnull()) & (df['血糖_'].notnull()) & (df['脂質_'].notnull())         # 3項目がある(nanがない)
                       , 's/o_MetS')                                                                    # 腹囲がある & 1つの項目が異常

                .mask((df['腹囲_'] == False) |                                                           # 腹囲正常　or
                     ((df['腹囲_'] == True) & (df['血圧_'] + df['血糖_'] + df['脂質_'] == 0) &             # 3項目が正常 &
                      (df['血圧_'].notnull()) & (df['血糖_'].notnull()) & (df['脂質_'].notnull()))        # 3項目がある(nanがない)
                       , 'not_MetS')                                                                    # 腹囲がない & 異常なし

                .mask((df['腹囲_'].isnull()) |                                                           # 腹囲がない or
                     ((df['腹囲_'] == True) &                                                            # 腹囲がある &
                     ((df['血圧_'].isnull()) & (df['血糖_'].isnull()) & (df['脂質_'].isnull()) |          # 3項目がない(nanがある) or
                      (df['血圧_'] + df['血糖_'] + df['脂質_'] == 1) &                                    # 1つの項目が異常 &
                     ((df['血圧_'].isnull()) | (df['血糖_'].isnull()) | (df['脂質_'].isnull()))))         # nanがある
                       , 'N/A')                                                                         # 判定なし                 
    )
    
    df_metS = df[df['MetS'] == 'MetS']
    df_so_metS = df[df['MetS'] == 's/o_MetS']
    df_non_metS = df[df['MetS'] == 'not_MetS']
    df_na_metS = df[df['MetS'] == 'N/A']
    return df_metS, df_so_metS, df_non_metS, df_na_metS

1   1,  1,  1,      MetS 　         1-1

1   1,  1,  nan     MetS            1-2

1   1,  0,  nan     can't 　        4-3

1   1,  nan,nan     can't 　        4-3

1   nan,nan,nan     can't 　        4-2

1   1,  0,  0       s/o 　          2-1
 　
0   1,  1,  1       not 　          3-1

0   nan,nan,nan     not 　          3-2

nan ~,  ~,  ~       can't           4-1


In [ ]:
#　変数なし

df = main.copy()

def mets_judge(df):   # 変数最小　#修正前
    #腹囲判定
    df['腹囲_'] = (df['腹囲'].mask((df['性別'] == '男') & (df['腹囲'] >= 85), True) # 男性       #どちらもデータがあってTrueの場合True       
                            .mask((df['性別'] == '女') & (df['腹囲'] >= 90), True) # 女性    　
                            .mask((df['性別'] == '男') & (df['腹囲'] < 85), False)  # 男性      #どちらもデータがあってFalseの場合False
                            .mask((df['性別'] == '女') & (df['腹囲'] < 90), False)  # 女性
                            .mask((df['性別'].isnull()) | (df['腹囲'].isnull()), np.nan)    #性別 or 腹囲がない場合nan
       )

    #血圧判定
    df['血圧_'] = (df['収縮期血圧'] .mask((df['収縮期血圧'] >= 130) | (df['拡張期血圧'] >= 85), True)      # どちらかがTrueの場合True
                              .mask((df['収縮期血圧'] < 130) & (df['拡張期血圧'] < 85), False)            # どちらもFalseの場合False
                              .mask((~(df['収縮期血圧'] >= 130) & ~(df['拡張期血圧'] >= 85))                    # どちらもFalseかnanであって
                                  & ((df['収縮期血圧'].isnull()) | (df['拡張期血圧'].isnull())), np.nan)      # どちらもnanの場合nan
       )

    #血糖判定
    df['血糖_'] = (df['空腹時血糖'].mask((df['空腹時血糖'] >= 110) | (df['HbA1c'] >= 6.0), True)          # どちらかがTrueの場合True
                              .mask((df['空腹時血糖'] < 110) & (df['HbA1c'] < 6.0), False)                # どちらもFalseの場合False
                              .mask((~(df['空腹時血糖'] >= 110) & ~(df['HbA1c'] >= 6.0))                   # どちらもFalseかnanであって
                                  & ((df['空腹時血糖'].isnull()) | (df['HbA1c'].isnull())), np.nan)      # どちらもnanの場合nan
       )

    #脂質判定
    df['脂質_'] = (df['中性脂肪'].mask((df['中性脂肪'] >= 150) | (df['HDL'] <= 40), True)           # どちらかがTrueの場合True
                               .mask((df['中性脂肪'] < 150) & (df['HDL'] > 40), False)               # どちらもFalseの場合False
                               .mask((~(df['中性脂肪'] >= 150) & ~(df['HDL'] <= 40))                 # どちらもFalseかnanであって     
                                  & ((df['中性脂肪'].isnull()) | (df['HDL'].isnull())), np.nan)      # どちらもnanの場合nan
       )

    df['MetS'] = (df['腹囲'] 
                .mask((df['腹囲_'].isnull()) |                                                           # 腹囲がない or                      # 4-1
                     ((df['腹囲_'] == True) &                                                            # 腹囲がある &
                     ((df['血圧_'].isnull()) & (df['血糖_'].isnull()) & (df['脂質_'].isnull()) |          # 3項目がない(nanがある) or           # 4-2 
                      (df['血圧_'] + df['血糖_'] + df['脂質_'] == 1) &                                    # 1つの項目が異常 &
                     ((df['血圧_'].isnull()) | (df['血糖_'].isnull()) | (df['脂質_'].isnull()))))         # nanがある                         # 4-3
                       , 'N/A')                                                                         # 判定なし         

                .mask((df['腹囲_'] == False) |                                                           # 腹囲正常　or                       # 3-1
                     ((df['腹囲_'] == True) &                                                            # 腹囲異常　&
                      (df['血圧_'] + df['血糖_'] + df['脂質_'] == 0) &                                    # 3項目が正常 &
                      (df['血圧_'].notnull()) & (df['血糖_'].notnull()) & (df['脂質_'].notnull()))        # 3項目がある(nanがない)              # 3-2
                       , 'not_MetS')                                                                    # 腹囲がない & 異常なし

                .mask((df['腹囲_'] == True) &                                                            # 腹囲異常　& 
                      (df['血圧_'] + df['血糖_'] + df['脂質_'] == 1) &                                    # 1つの項目が異常 &
                      (df['血圧_'].notnull()) & (df['血糖_'].notnull()) & (df['脂質_'].notnull())         # 3項目がある(nanがない)              # 2-1
                       , 's/o_MetS')                                                                    # 腹囲がある & 1つの項目が異常
                  
                .mask((df['腹囲_'] == True) & ((df['血圧_'] + df['血糖_'] + df['脂質_'] >= 2))             # 1-1, 1-2
                       , 'MetS')                                                                        # 腹囲がある & 2つ以上の項目が異常                                                                     
       )
    
    df_metS = df[df['MetS'] == 'MetS']
    df_so_metS = df[df['MetS'] == 's/o_MetS']
    df_non_metS = df[df['MetS'] == 'not_MetS']
    df_na_metS = df[df['MetS'] == 'N/A']
    return df_metS, df_so_metS, df_non_metS, df_na_metS

In [ ]:
mets_judge(df)
df

In [ ]:
# 実行
df_metS, df_so_metS, df_non_metS, df_na_metS = mets_judge(df)
print(df_metS)
print(df_non_metS)

In [ ]:
df.isna().sum()

In [ ]:
# facter_binarization
def facter_binarization(df, facter_th = ['HbA1c', '>= 5.6']):        #カテゴリー変数を２値化、trueが１、true_valueはリスト
    df[f'{facter_th[0]}_'] = list(eval(f"(df['{facter_th[0]}']\
                                       .mask(df['{facter_th[0]}'] {facter_th[1]}, 1)\
                                       .mask(~(df['{facter_th[0]}'] {facter_th[1]}), 0)\
                                       .mask(df['{facter_th[0]}'].isnull(), np.nan))"
                                       ))

## 12/18
- 年度で数字を振り分ける
- ハッシュ値で横並びに

In [ ]:
# import master_code as mc
import numpy as np
import pandas as pd
main = pd.read_csv('/Volumes/USBDISK/AHHDC/git/html/dummy2.csv', encoding = 'cp932', index_col = 0)
#　変数なし

df = main.copy()

def mets_judge(df_):   # メタボの判定
   #腹囲判定
   df = df_.copy()
   print(df)
   df['腹囲_'] = (df['腹囲'].mask((df['性別'] == '男') & (df['腹囲'] >= 85), 1) # 男性       #どちらもデータがあってTrueの場合True       
                           .mask((df['性別'] == '女') & (df['腹囲'] >= 90), 1) # 女性    　
                           .mask((df['性別'] == '男') & (df['腹囲'] < 85), 0)  # 男性      #どちらもデータがあってFalseの場合False
                           .mask((df['性別'] == '女') & (df['腹囲'] < 90), 0)  # 女性
                           .mask((df['性別'].isnull()) | (df['腹囲'].isnull()), np.nan)    #性別 or 腹囲がない場合nan
      )

   #血圧判定
   df['血圧_'] = (df['収縮期血圧'] .mask((df['収縮期血圧'] >= 130) | (df['拡張期血圧'] >= 85), 1)      # どちらかがTrueの場合True
                           .mask((df['収縮期血圧'] < 130) & (df['拡張期血圧'] < 85), 0)            # どちらもFalseの場合False
                           .mask((~(df['収縮期血圧'] >= 130) & ~(df['拡張期血圧'] >= 85))                    # どちらもFalseかnanであって
                                 & ((df['収縮期血圧'].isnull()) | (df['拡張期血圧'].isnull())), np.nan)      # どちらもnanの場合nan
      )

   #血糖判定
   df['血糖_'] = (df['空腹時血糖'].mask((df['空腹時血糖'] >= 110) | (df['HbA1c'] >= 6.0), 1)          # どちらかがTrueの場合True
                           .mask((df['空腹時血糖'] < 110) & (df['HbA1c'] < 6.0), 0)                # どちらもFalseの場合False
                           .mask((~(df['空腹時血糖'] >= 110) & ~(df['HbA1c'] >= 6.0))                   # どちらもFalseかnanであって
                                 & ((df['空腹時血糖'].isnull()) | (df['HbA1c'].isnull())), np.nan)      # どちらもnanの場合nan
      )

   #脂質判定
   df['脂質_'] = (df['中性脂肪'].mask((df['中性脂肪'] >= 150) | (df['HDL'] <= 40), 1)           # どちらかがTrueの場合True
                              .mask((df['中性脂肪'] < 150) & (df['HDL'] > 40), 0)               # どちらもFalseの場合False
                              .mask((~(df['中性脂肪'] >= 150) & ~(df['HDL'] <= 40))                 # どちらもFalseかnanであって     
                                 & ((df['中性脂肪'].isnull()) | (df['HDL'].isnull())), np.nan)      # どちらもnanの場合nan
      )

   df['MetS'] = (df['腹囲'] 
               .mask((df['腹囲_'].isnull()) |                                                           # 腹囲がない or                      # 4-1
                  ((df['腹囲_'] == 1) &                                                               # 腹囲がある &
                  ((df['血圧_'].isnull()) & (df['血糖_'].isnull()) & (df['脂質_'].isnull()) |          # 3項目がない(nanがある) or           # 4-2 
                  ((df[['腹囲_', '血圧_', '血糖_', '脂質_']].sum(axis = 1) <= 1) &                                    # 1つの項目が異常 &
                  ((df['血圧_'].isnull()) | (df['血糖_'].isnull()) | (df['脂質_'].isnull())))))         # nanがある                         # 4-3
                     , 'N/A')                                                                         # 判定なし         

               .mask((df['腹囲_'] == 0) |                                                               # 腹囲正常　or                       # 3-1
                  ((df['腹囲_'] == 1) &                                                               # 腹囲異常　&
                     (df[['腹囲_', '血圧_', '血糖_', '脂質_']].sum(axis = 1) == 0) &                                    # 3項目が正常 &
                     (df['血圧_'].notnull()) & (df['血糖_'].notnull()) & (df['脂質_'].notnull()))        # 3項目がある(nanがない)              # 3-2
                     , 'not_MetS')                                                                    # 腹囲がない & 異常なし

               .mask((df['腹囲_'] == 1) &                                                               # 腹囲異常　& 
                     (df[['腹囲_', '血圧_', '血糖_', '脂質_']].sum(axis = 1) == 1) &                                    # 1つの項目が異常 &
                     (df['血圧_'].notnull()) & (df['血糖_'].notnull()) & (df['脂質_'].notnull())         # 3項目がある(nanがない)              # 2-1
                     , 's/o_MetS')                                                                    # 腹囲がある & 1つの項目が異常
               
               .mask(((df['腹囲_'] == 1) & (df[['腹囲_', '血圧_', '血糖_', '脂質_']].sum(axis = 1) >= 2))                # 1-1, 1-2
                     , 'MetS')                                                                        # 腹囲がある & 2つ以上の項目が異常                                                                     
   )
   df_metS = df[df['MetS'] == 'MetS']
   df_so_metS = df[df['MetS'] == 's/o_MetS']
   df_non_metS = df[df['MetS'] == 'not_MetS']
   df_na_metS = df[df['MetS'] == 'N/A']
   return df, df_metS, df_so_metS, df_non_metS, df_na_metS

def merge_df_year(df, years_name = '検診実施年度'):      # 検診実施年度をキーにして結合
   implementation_years = sorted(list(df[years_name].unique()))                                        # 検診実施年度のリスト
   for i, implementation_year in enumerate(implementation_years):
      df_1 = (df[df[years_name] == implementation_year]                                           # 検診実施年度でフィルター
               .drop(columns = years_name)                                                         # 検診実施年度を削除
               .rename(columns = lambda x: x if x in ['ID',] else x + f'_{implementation_year}'))    # ID以外の列名に検診実施年度を付与
      if i == 0:
         merge_df = df_1                                                                           # 初回はそのまま代入
      else:
         merge_df = pd.merge(merge_df, df_1, on=['ID', ], how='outer')   #inner?                   # 2回目以降はIDをキーにして結合
   return merge_df



In [ ]:
df = main.copy()
df, df_metS, df_so_metS, df_non_metS, df_na_metS = mets_judge(df)
merge_df = merge_df_year(df)
# マージまで
# 何をするか
#print(len(merge_df['ID'].unique()))
#merge_df

## 変化を見る

In [ ]:
#前回まで
# import master_code as mc
import numpy as np
import pandas as pd
main = pd.read_csv('/Volumes/USBDISK/AHHDC/git/html/dummy2.csv', encoding = 'cp932', index_col = 0)

def mets_judge(df_):   # メタボの判定
   #腹囲判定
   df = df_.copy()
   print(df)
   df['腹囲_'] = (df['腹囲'].mask((df['性別'] == '男') & (df['腹囲'] >= 85), 1) # 男性       #どちらもデータがあってTrueの場合True       
                           .mask((df['性別'] == '女') & (df['腹囲'] >= 90), 1) # 女性    　
                           .mask((df['性別'] == '男') & (df['腹囲'] < 85), 0)  # 男性      #どちらもデータがあってFalseの場合False
                           .mask((df['性別'] == '女') & (df['腹囲'] < 90), 0)  # 女性
                           .mask((df['性別'].isnull()) | (df['腹囲'].isnull()), np.nan)    #性別 or 腹囲がない場合nan
      )

   #血圧判定
   df['血圧_'] = (df['収縮期血圧'] .mask((df['収縮期血圧'] >= 130) | (df['拡張期血圧'] >= 85), 1)      # どちらかがTrueの場合True
                           .mask((df['収縮期血圧'] < 130) & (df['拡張期血圧'] < 85), 0)            # どちらもFalseの場合False
                           .mask((~(df['収縮期血圧'] >= 130) & ~(df['拡張期血圧'] >= 85))                    # どちらもFalseかnanであって
                                 & ((df['収縮期血圧'].isnull()) | (df['拡張期血圧'].isnull())), np.nan)      # どちらもnanの場合nan
      )

   #血糖判定
   df['血糖_'] = (df['空腹時血糖'].mask((df['空腹時血糖'] >= 110) | (df['HbA1c'] >= 6.0), 1)          # どちらかがTrueの場合True
                           .mask((df['空腹時血糖'] < 110) & (df['HbA1c'] < 6.0), 0)                # どちらもFalseの場合False
                           .mask((~(df['空腹時血糖'] >= 110) & ~(df['HbA1c'] >= 6.0))                   # どちらもFalseかnanであって
                                 & ((df['空腹時血糖'].isnull()) | (df['HbA1c'].isnull())), np.nan)      # どちらもnanの場合nan
      )

   #脂質判定
   df['脂質_'] = (df['中性脂肪'].mask((df['中性脂肪'] >= 150) | (df['HDL'] <= 40), 1)           # どちらかがTrueの場合True
                              .mask((df['中性脂肪'] < 150) & (df['HDL'] > 40), 0)               # どちらもFalseの場合False
                              .mask((~(df['中性脂肪'] >= 150) & ~(df['HDL'] <= 40))                 # どちらもFalseかnanであって     
                                 & ((df['中性脂肪'].isnull()) | (df['HDL'].isnull())), np.nan)      # どちらもnanの場合nan
      )

   df['MetS'] = (df['腹囲'] 
               .mask((df['腹囲_'].isnull()) |                                                           # 腹囲がない or                      # 4-1
                  ((df['腹囲_'] == 1) &                                                               # 腹囲がある &
                  ((df['血圧_'].isnull()) & (df['血糖_'].isnull()) & (df['脂質_'].isnull()) |          # 3項目がない(nanがある) or           # 4-2 
                  ((df[['腹囲_', '血圧_', '血糖_', '脂質_']].sum(axis = 1) <= 1) &                                    # 1つの項目が異常 &
                  ((df['血圧_'].isnull()) | (df['血糖_'].isnull()) | (df['脂質_'].isnull())))))         # nanがある                         # 4-3
                     , 'N/A')                                                                         # 判定なし         

               .mask((df['腹囲_'] == 0) |                                                               # 腹囲正常　or                       # 3-1
                  ((df['腹囲_'] == 1) &                                                               # 腹囲異常　&
                     (df[['腹囲_', '血圧_', '血糖_', '脂質_']].sum(axis = 1) == 0) &                                    # 3項目が正常 &
                     (df['血圧_'].notnull()) & (df['血糖_'].notnull()) & (df['脂質_'].notnull()))        # 3項目がある(nanがない)              # 3-2
                     , 'not_MetS')                                                                    # 腹囲がない & 異常なし

               .mask((df['腹囲_'] == 1) &                                                               # 腹囲異常　& 
                     (df[['腹囲_', '血圧_', '血糖_', '脂質_']].sum(axis = 1) == 1) &                                    # 1つの項目が異常 &
                     (df['血圧_'].notnull()) & (df['血糖_'].notnull()) & (df['脂質_'].notnull())         # 3項目がある(nanがない)              # 2-1
                     , 's/o_MetS')                                                                    # 腹囲がある & 1つの項目が異常
               
               .mask(((df['腹囲_'] == 1) & (df[['腹囲_', '血圧_', '血糖_', '脂質_']].sum(axis = 1) >= 2))                # 1-1, 1-2
                     , 'MetS')                                                                        # 腹囲がある & 2つ以上の項目が異常                                                                     
   )
   df_metS = df[df['MetS'] == 'MetS']
   df_so_metS = df[df['MetS'] == 's/o_MetS']
   df_non_metS = df[df['MetS'] == 'not_MetS']
   df_na_metS = df[df['MetS'] == 'N/A']
   return df, df_metS, df_so_metS, df_non_metS, df_na_metS

def merge_df_year(df, years_name = '検診実施年度'):      # 検診実施年度をキーにして結合
   implementation_years = sorted(list(df[years_name].unique()))                                        # 検診実施年度のリスト
   for i, implementation_year in enumerate(implementation_years):
      df_1 = (df[df[years_name] == implementation_year]                                           # 検診実施年度でフィルター
               .drop(columns = years_name)                                                         # 検診実施年度を削除
               .rename(columns = lambda x: x if x in ['ID',] else x + f'_{implementation_year}'))    # ID以外の列名に検診実施年度を付与
      if i == 0:
         merge_df = df_1                                                                           # 初回はそのまま代入
      else:
         merge_df = pd.merge(merge_df, df_1, on=['ID', ], how='outer')   #inner?                   # 2回目以降はIDをキーにして結合
   return merge_df

df = main.copy()
df, df_metS, df_so_metS, df_non_metS, df_na_metS = mets_judge(df)
merge_df = merge_df_year(df)

In [2]:
def make_pair(target_list, delta):#リストから指定の間隔をあけて二つを取り出す
    pairs = [[target_list[i], target_list[i + delta]] for i in range(len(target_list) - delta)]
    return pairs

In [3]:
#df = merge_df.copy()
years_name = '検診実施年度'
years = sorted(list(df[years_name].unique()))
target_years = make_pair(years, 2)[0]    # [2016, 2018]を取得
df_target= merge_df[(merge_df[f'MetS_{target_years[0]}'] == 'not_MetS') &                                                  # 2016年がnot_MetSのデータを抽出
             ((merge_df[f'MetS_{target_years[1]}'] == 'MetS') |
              (merge_df[f'MetS_{target_years[1]}'] == 'not_MetS') |
              (merge_df[f'MetS_{target_years[1]}'] == 's/o_MetS'))]                                                      
#判定
df_target.loc[((df_target[f'MetS_{target_years[0]}'] == 'not_MetS') &     # 2016年がnot_MetS & 2018年がMetSの場合1
              ((df_target[f'MetS_{target_years[1]}'] == 'MetS') | (df_target[f'MetS_{target_years[1]}'] == 's/o_MetS'))), 
              '判定'] = 1    
df_target.loc[((df_target[f'MetS_{target_years[0]}'] == 'not_MetS') &     # 2016年がnot_MetS & 2018年がnot_MetSの場合0
               (df_target[f'MetS_{target_years[1]}'] == 'not_MetS')), 
               '判定'] = 0                 
df_target.drop(columns = df_target.columns[~df_target.columns.str.endswith((f'{target_years[0]}','判定'))], inplace = True) #古いデータを削除

df_target



/var/folders/r0/3xgnd3jn663c7fl8x27td0q40000gn/T/ipykernel_1324/1006020693.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_target.loc[((df_target[f'MetS_{target_years[0]}'] == 'not_MetS') &     # 2016年がnot_MetS & 2018年がMetSの場合1
/var/folders/r0/3xgnd3jn663c7fl8x27td0q40000gn/T/ipykernel_1324/1006020693.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_target.drop(columns = df_target.columns[~df_target.columns.str.endswith((f'{target_years[0]}','判定'))], inplace = True) #古いデータを削除


,市町村名_2016,性別_2016,年齢_2016,腹囲_2016,空腹時血糖_2016,HbA1c_2016,収縮期血圧_2016,拡張期血圧_2016,中性脂肪_2016,HDL_2016,...,脂質服薬_2016,喫煙_2016,睡眠_2016,運動習慣_2016,腹囲__2016,血圧__2016,血糖__2016,脂質__2016,MetS_2016,判定
0,C市,女,70.0,70.5,96.0,5.2,96.8,60.0,54.4,61.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,not_MetS,0.0
2,C市,女,72.0,89.7,88.0,5.7,118.4,96.0,95.2,56.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,not_MetS,0.0
3,C市,女,61.0,87.0,89.0,5.2,168.0,63.2,104.4,56.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,not_MetS,0.0
22,C市,女,67.0,78.0,116.0,6.3,102.4,64.0,169.2,59.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,not_MetS,0.0
43,C市,女,72.0,83.0,108.0,6.2,99.2,96.0,44.8,49.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,not_MetS,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6629,A市,女,68.0,79.9,118.0,6.5,111.2,100.8,103.2,84.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,not_MetS,0.0
6667,A市,女,67.0,69.0,87.0,5.6,120.0,84.0,96.0,64.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,not_MetS,0.0
6674,A市,女,69.0,81.0,131.0,6.8,153.6,56.0,87.2,49.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,not_MetS,0.0
6680,A市,女,71.0,82.0,92.0,6.2,107.2,74.4,131.2,50.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,not_MetS,0.0


In [7]:
df_target

,市町村名_2016,性別_2016,年齢_2016,腹囲_2016,空腹時血糖_2016,HbA1c_2016,収縮期血圧_2016,拡張期血圧_2016,中性脂肪_2016,HDL_2016,...,脂質服薬_2016,喫煙_2016,睡眠_2016,運動習慣_2016,腹囲__2016,血圧__2016,血糖__2016,脂質__2016,MetS_2016,判定
0,C市,女,70.0,70.5,96.0,5.2,96.8,60.0,54.4,61.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,not_MetS,0.0
2,C市,女,72.0,89.7,88.0,5.7,118.4,96.0,95.2,56.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,not_MetS,0.0
3,C市,女,61.0,87.0,89.0,5.2,168.0,63.2,104.4,56.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,not_MetS,0.0
22,C市,女,67.0,78.0,116.0,6.3,102.4,64.0,169.2,59.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,not_MetS,0.0
43,C市,女,72.0,83.0,108.0,6.2,99.2,96.0,44.8,49.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,not_MetS,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6629,A市,女,68.0,79.9,118.0,6.5,111.2,100.8,103.2,84.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,not_MetS,0.0
6667,A市,女,67.0,69.0,87.0,5.6,120.0,84.0,96.0,64.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,not_MetS,0.0
6674,A市,女,69.0,81.0,131.0,6.8,153.6,56.0,87.2,49.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,not_MetS,0.0
6680,A市,女,71.0,82.0,92.0,6.2,107.2,74.4,131.2,50.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,not_MetS,0.0


## モデル？ 

必要なもの
- カラムの組み合わせ
- データの下見

In [4]:
column_names = df_target.columns.to_list()
column_names.remove('判定')
column_names

['市町村名_2016',
 '性別_2016',
 '年齢_2016',
 '腹囲_2016',
 '空腹時血糖_2016',
 'HbA1c_2016',
 '収縮期血圧_2016',
 '拡張期血圧_2016',
 '中性脂肪_2016',
 'HDL_2016',
 'LDL_2016',
 '血糖服薬_2016',
 '血圧服薬_2016',
 '脂質服薬_2016',
 '喫煙_2016',
 '睡眠_2016',
 '運動習慣_2016',
 '腹囲__2016',
 '血圧__2016',
 '血糖__2016',
 '脂質__2016',
 'MetS_2016']

In [8]:
# モデル作成
import statsmodels.api as sm
import statsmodels.formula.api as smf
model = smf.ols(formula = '判定 ~ 血圧__2016 + 血糖__2016', data = df_target).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                     判定   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.2891
Date:                Wed, 10 Jan 2024   Prob (F-statistic):              0.749
Time:                        16:47:26   Log-Likelihood:                -29.562
No. Observations:                 263   AIC:                             65.12
Df Residuals:                     260   BIC:                             75.84
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0651      0.030      2.203      0.0

In [ ]:
model.summary().tables[1]

In [ ]:
model.summary().tables[1]

In [9]:
model.aic

65.12441918622898

In [ ]:
#多変量解析　ステップワイズ
def stepAIC(df, exp_list, exp_list_plus, target):#コンマ刻みのリストを渡す
    import copy
    descriptors = exp_list
    g_model,  best_aic = glm_s(df, exp_list_plus, target)
    best_model = g_model
    while descriptors:
        desc_selected = ''
        flag = 0
        for desk in descriptors:
            used_desks = copy.deepcopy(descriptors)
            used_desks.remove(desk)
            formula =  ' + '.join(used_desks)
            model = glm_s(df, formula, target)
            if model.aic < best_aic:
                best_aic = model.aic
                best_model = model
                desc_selected = desk
                flag = 1
        if flag:
            descriptors.remove(desc_selected)
        else:
            break
    return best_model, resAIC
